In [1]:
import findspark
findspark.init()
findspark.find()


'/Users/chiholee/02.Project/spark/spark'

In [2]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Read S3 Parquet1") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 20:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_bigquery = spark.read \
    .format("parquet") \
    .option("header", True) \
    .load("data/run-1679317775429-part-block-0-r-00000-uncompressed.parquet")


23/04/02 20:37:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [27]:
df_bigquery.printSchema()
# df_bigquery.show()
# df_bigquery.describe().show()


root
 |-- visitorId: long (nullable = true)
 |-- visitNumber: long (nullable = true)
 |-- visitId: long (nullable = true)
 |-- visitStartTime: long (nullable = true)
 |-- date: string (nullable = true)
 |-- totals: struct (nullable = true)
 |    |-- visits: long (nullable = true)
 |    |-- hits: long (nullable = true)
 |    |-- pageviews: long (nullable = true)
 |    |-- timeOnSite: long (nullable = true)
 |    |-- bounces: long (nullable = true)
 |    |-- transactions: long (nullable = true)
 |    |-- transactionRevenue: long (nullable = true)
 |    |-- newVisits: long (nullable = true)
 |    |-- screenviews: long (nullable = true)
 |    |-- uniqueScreenviews: long (nullable = true)
 |    |-- timeOnScreen: long (nullable = true)
 |    |-- totalTransactionRevenue: long (nullable = true)
 |    |-- sessionQualityDim: long (nullable = true)
 |-- trafficSource: struct (nullable = true)
 |    |-- referralPath: string (nullable = true)
 |    |-- campaign: string (nullable = true)
 |    |-- s

In [5]:
df_bigquery.createOrReplaceTempView("bigquery")

In [6]:
spark.sql("""
SELECT
  COUNT(DISTINCT fullVisitorId) as total_visitors
FROM bigquery
""").show(truncate=False)

+--------------+
|total_visitors|
+--------------+
|127           |
+--------------+



In [7]:
spark.sql("""
SELECT
  *
FROM
  bigquery a  
LIMIT 1
""").show(truncate=False)

+---------+-----------+----------+--------------+--------+------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------

In [30]:
spark.sql("""
SELECT
  a.visitNumber,
  a.visitId,
  a.visitStartTime,
  a.date,
  totals.visits,
  totals.hits,
  col_hits.type,
  col_hits.page.pagePath AS page_path  
FROM
  bigquery a
  lateral view explode(hits) as col_hits 
LIMIT
  10	
""").show(truncate=False)

+-----------+----------+--------------+--------+------+----+----+--------------------------------------+
|visitNumber|visitId   |visitStartTime|date    |visits|hits|type|page_path                             |
+-----------+----------+--------------+--------+------+----+----+--------------------------------------+
|1          |1501620894|1501620894    |20170801|1     |27  |PAGE|/home                                 |
|1          |1501620894|1501620894    |20170801|1     |27  |PAGE|/home                                 |
|1          |1501620894|1501620894    |20170801|1     |27  |PAGE|/home                                 |
|1          |1501620894|1501620894    |20170801|1     |27  |PAGE|/google+redesign/shop+by+brand/youtube|
|1          |1501620894|1501620894    |20170801|1     |27  |PAGE|/google+redesign/shop+by+brand/youtube|
|1          |1501620894|1501620894    |20170801|1     |27  |PAGE|/home                                 |
|1          |1501620894|1501620894    |20170801|1     |